In [ ]:
# Import required libraries
import pandas as pd
from geopy.geocoders import Nominatim  # For reverse geocoding (getting address from coordinates)

# Define the file path to the Excel workbook containing location data
file_path = '.\Downloads\\Forggith.xlsx'  # Update this path based on your local setup

# Specify the sheet name that contains the location information
sheet_name = "DimLocation"  # This should match the exact sheet name in the Excel file

# Load the specified Excel sheet into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Preview the first few rows of the DataFrame to confirm successful load
df.head()


<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP\AppData\Local\Temp\ipykernel_18636\943719071.py:3: SyntaxWarning: invalid escape sequence '\D'
  file_path = 'C:\\Users\\HP\Downloads\Forggith.xlsx' # change this to your actual file path


,LocationID,City,Latitude,Longitude
0,Ditzingen-48.8264-9.0667,Ditzingen,48.8264,9.0667
1,Backnang-48.9464-9.4306,Backnang,48.9464,9.4306
2,Rheinbach-50.6256-6.9491,Rheinbach,50.6256,6.9491
3,Fürth-49.4783-10.9903,Fürth,49.4783,10.9903
4,Geldern-51.5197-6.3325,Geldern,51.5197,6.3325


In [ ]:
# Testing geopy with a sample pair of latitude and longitude

# Initialize the Nominatim geolocator with a custom user agent (recommended to use your email)
geolocator = Nominatim(user_agent="lbeerus2025@gmail.com")

# Perform reverse geocoding on a sample coordinate (latitude, longitude)
location = geolocator.reverse("47.5611, 7.7917")

# Print the full address returned from the coordinate
print(location.address)


Pfarrer-Herrmann-Weg, Nollingen, Rheinfelden (Baden), Verwaltungsgemeinschaft Rheinfelden (Baden), Landkreis Lörrach, Baden-Württemberg, 79618, Deutschland


In [ ]:
# Testing geopy with a real pair of latitude and longitude from the dataset

# Format the first row's latitude and longitude into a string "lat, lon"
coordinates = f"{df['Latitude'].iloc[0]}, {df['Longitude'].iloc[0]}"

# Use geopy's reverse geocoding to get the address of that coordinate
location1 = geolocator.reverse(coordinates)

# Print the resulting address to inspect what was returned
print(location1.address)


Rathaus, 1, Am Laien, Brühl, Ditzingen, Landkreis Ludwigsburg, Baden-Württemberg, 71254, Deutschland


In [ ]:
# Since the dataset has only about 500 rows, I didnt set a timeout
# Initialize the geolocator using Nominatim with a custom user agent (use your email or app name)
geolocator = Nominatim(user_agent="lbeerus2025@gmail.com")  # Replace with your email or application name

# Define a function to reverse geocode each row's coordinates into a human-readable address
def reverse_geocode(row):
    try:
        # Combine Latitude and Longitude values from the row into a "lat, lon" string
        coordinates = f"{row['Latitude']}, {row['Longitude']}"
        
        # Use geopy to get the location object corresponding to the coordinates
        location = geolocator.reverse(coordinates, exactly_one=True)
        
        # Return the address if found, else return None
        if location and location.address:
            return location.address
        else:
            return None
    except Exception as e:
        # If an error occurs (e.g., rate limiting, connection issue), print it and return None
        print(f"Error reverse geocoding: {e}")
        return None

# Apply the reverse_geocode function to each row to generate an Address column
df['Address'] = df.apply(reverse_geocode, axis=1)

# Display the first few results to verify that addresses were successfully retrieved
print(df[['Latitude', 'Longitude', 'Address']].head())


Error reverse geocoding: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=47.5611&lon=7.7917&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000203295AF6E0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
   Latitude  Longitude                                            Address
0   48.8264     9.0667  Rathaus, 1, Am Laien, Brühl, Ditzingen, Landkr...
1   48.9464     9.4306  zauberhaft!, Kuhbergele, Backnang, Vereinbarte...
2   50.6256     6.9491  27, Hauptstraße, Rheinbach, Rhein-Sieg-Kreis, ...
3   49.4783    10.9903  6, Gustavstraße, Altstadt, Fürth, Bayern, 9076...
4   51.5197     6.3325  Herr Dr. med. Stefan Pingsten, 1, Stauffenberg...


In [ ]:
# Code with time out and  increased max retries
# Importing necessary modules from geopy and standard libraries
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

# Initialize the Nominatim geocoder with a user agent and increased timeout
geolocator = Nominatim(user_agent="your_app_name", timeout=10)  # Replace "your_app_name" with your actual identifier

# Function to reverse geocode with retry mechanism in case of timeouts or service errors
def reverse_geocode_with_retry(row, max_retries=3, retry_delay=2):
    for attempt in range(max_retries):
        try:
            # Construct the coordinate string from the Latitude and Longitude in the row
            coordinates = f"{row['Latitude']}, {row['Longitude']}"
            
            # Attempt to get the address from the coordinates
            location = geolocator.reverse(coordinates, exactly_one=True)
            
            # If a valid location is found, return its address
            if location and location.address:
                return location.address
            else:
                return None
        
        # Handle expected geocoding exceptions and retry if applicable
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Geocoding failed (attempt {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # Wait before retrying
            continue
        
        # Handle any other unexpected exceptions
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None
    
    # Return None if all retries fail
    return None

# Apply the reverse geocoding function across each row in the DataFrame
df['Address'] = df.apply(reverse_geocode_with_retry, axis=1)

# Print the first few results to verify success
print(df[['Latitude', 'Longitude', 'Address']].head())


In [ ]:
# Viewing the 'Address' column to inspect the full reverse-geocoded results
# This helps verify the structure and consistency of the returned location strings
df['Address']

0      Rathaus, 1, Am Laien, Brühl, Ditzingen, Landkr...
1      zauberhaft!, Kuhbergele, Backnang, Vereinbarte...
2      27, Hauptstraße, Rheinbach, Rhein-Sieg-Kreis, ...
3      6, Gustavstraße, Altstadt, Fürth, Bayern, 9076...
4      Herr Dr. med. Stefan Pingsten, 1, Stauffenberg...
                             ...                        
546    22, Heinrich-Heine-Platz, Mitte, Kernstadt, La...
547    Oberallgäuer Platz, Illersiedlung, Margarethen...
548    JOH, 15 - 21, Hauptstraße, Altstadt Zwickau, I...
549    2, Erich-Weinert-Straße, Wolkenrasen, Oberlind...
550    10, Markt, Innenstadt, Helmstedt, Landkreis He...
Name: Address, Length: 551, dtype: object

In [ ]:
# Function to extract the State from the 'Address' column
def extract_state(address):
    """
    Extracts the third element from the end of a comma-separated address string,
    which is assumed to be the state.

    Args:
        address (str): Full address returned by reverse geocoding.

    Returns:
        str or None: Extracted state name, or None if the address is invalid or too short.
    """
    if address is None:
        return None
    
    # Split the address into parts using comma as a separator
    parts = address.split(', ')
    
    # Return the third element from the right if enough parts exist
    if len(parts) >= 3:
        return parts[-3]
    else:
        return None

# Apply the function to the DataFrame to extract and create a new 'State' column
df['State'] = df['Address'].apply(extract_state)

# Display the first few rows to verify the extraction
print(df[['Address', 'State']].head())


                                             Address                State
0  Rathaus, 1, Am Laien, Brühl, Ditzingen, Landkr...    Baden-Württemberg
1  zauberhaft!, Kuhbergele, Backnang, Vereinbarte...    Baden-Württemberg
2  27, Hauptstraße, Rheinbach, Rhein-Sieg-Kreis, ...  Nordrhein-Westfalen
3  6, Gustavstraße, Altstadt, Fürth, Bayern, 9076...               Bayern
4  Herr Dr. med. Stefan Pingsten, 1, Stauffenberg...  Nordrhein-Westfalen


In [14]:
df['State'].value_counts().index

Index(['Nordrhein-Westfalen', 'Baden-Württemberg', 'Bayern', 'Niedersachsen',
       'Hessen', 'Rheinland-Pfalz', 'Brandenburg', 'Thüringen',
       'Sachsen-Anhalt', 'Sachsen', 'Schleswig-Holstein',
       'Mecklenburg-Vorpommern', 'Saarland', 'Bremen', 'Hamburg', 'Berlin'],
      dtype='object', name='State')

In [15]:
df.to_excel("location_states.xlsx", index=False)

 *A hierarchy can now be created using the State and City columns to enable better map visualizations that support drilldowns of revenue contributions and order counts*